In [46]:
%reload_ext autoreload
%autoreload 2
from SFM import SFM
import os, shutil
import pycolmap, platform
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
src = "datasets/templeRing"
dest = "colmap_test_out"
try:
    os.mkdir(dest)
except:
    shutil.rmtree(dest)
    os.mkdir(dest)

In [3]:
test = SFM(src)

00.png:   0%|          | 0/49 [00:00<?, ?it/s]

templeR_par.txt: 100%|██████████| 49/49 [00:00<00:00, 91.31it/s]


README.txt is not an img...

templeR_ang.txt is not an img...

templeR_par.txt is not an img...



In [4]:
test.ft_extract()
matches = test.ft_match()



FEATURE EXTRACTION....


Image 46: 100%|██████████| 46/46 [00:03<00:00, 14.10it/s]



MATCHING...



(44, 45): 100%|██████████| 45/45 [00:22<00:00,  2.01it/s]                         

Elapsed Time: 22.445398092269897




In [32]:
d = test.desc

dd = d[0][1][0].astype(np.int64).flatten()

temp = ' '.join([f"{num}" for num in dd])

f'x y {temp}'

'x y 4 52 134 111 134 134 68 15 22 4 38 134 134 21 11 29 7 6 29 94 41 13 7 6 2 8 14 16 20 16 6 2 21 72 134 97 108 39 6 16 5 12 29 106 123 38 15 5 10 17 20 26 31 30 15 8 2 9 13 21 26 20 3 2 41 12 19 64 71 13 7 25 9 19 29 72 84 31 10 10 11 21 32 36 41 31 11 7 1 5 19 33 27 14 2 1 20 7 22 71 65 8 3 8 3 7 28 62 35 5 1 1 2 20 33 31 19 15 5 1 1 4 16 20 13 11 7 2'

In [59]:
matches[(1,2)][0]

< cv2.DMatch 0x288497410>

In [51]:
# plt.imshow(test.adj_list()[0])
if not test.matches[(10,10)]:
    print("no")

no


In [ ]:
cv2.DMatch.trainIdx

In [63]:
def output_ft(sfm: SFM, desc: tuple, dest, i):
    if platform.system() == "Windows":
        path = f"{dest}\\features"
    else:
        path = f"{dest}/features"
    if i == 0:
        try:
            os.mkdir(path)
        except:
            shutil.rmtree(path)
            os.mkdir(path)

    im = sfm.itoimg[i]
    if platform.system() == "Windows":
        f = open(f"{path}\\{im}.txt", "w+")
    else:
        f = open(f"{path}/{im}.txt", "w+")

    n_ft = desc[i][1].shape[0]
    text = f"{n_ft} 128"

    for j in range(n_ft):
        x = desc[i][0][j].pt[0]
        y = desc[i][0][j].pt[1]
        scale = desc[i][0][j].size
        orientation = desc[i][0][j].angle

        D = desc[i][1][j].astype(np.int64).flatten()
        D_txt = ' '.join([f"{d}" for d in D])
        cur = f"\n{x} {y} {scale} {orientation} {D_txt}"
        text += cur
    
    f.write(text)
    f.close()



def output_match(sfm: SFM, desc: tuple, dest):
    if platform.system() == "Windows":
        path = f"{dest}\\matches"
    else:
        path = f"{dest}/matches"
    
    try:
        os.mkdir(path)
    except:
        shutil.rmtree(path)
        os.mkdir(path)

    if platform.system() == "Windows":
        f = open(f"{path}\\match.txt", "w+")
    else:
        f = open(f"{path}/match.txt", "w+")

    text = ""
    for i in range(sfm.n - 1):
        for j in range(i+1, sfm.n):
            if not sfm.good[(i, j)]:
                continue
                
            im1 = sfm.itoimg[i]
            im2 = sfm.itoimg[j]

            cur = f"{im1} {im2}"

            for m in sfm.good[(i,j)]:
                q = m.queryIdx
                t = m.trainIdx
                cur += f"\n{q} {t}"
            cur += f"\n\n"
            text += cur
            


    f.write(text)
    f.close()

    
    


In [64]:
output_match(test, test.desc, dest)

In [61]:
for i in range(test.n):
    output_ft(test, test.desc, dest, i)



UnboundLocalError: local variable 'i' referenced before assignment